In [1]:
import pandas
import pickle
import itertools
import re
import math
import numpy
from pg import DB
db = DB()
q = db.query("SELECT motif_string FROM motif")

In [18]:
db.query("ALTER TABLE motif ADD COLUMN bs_average_score float;")
db.query("ALTER TABLE motif ADD COLUMN bs_shuffled_average_score float;")

In [ ]:
%%time
i = 0
for motif_t in q.getresult():
    motif = motif_t[0]
    if(db.query("SELECT average_score FROM motif where motif_string = '" + motif +"' and bs_average_score NOTNULL").ntuples() > 0):
        continue
        
    q_result = db.query("SELECT subgraph,shuffled_subgraph FROM motif where motif_string = '" + motif + "'").getresult()[0]
    subgraph = q_result[0]
    avg_score = pandas.DataFrame(subgraph).mean().mean()
    ninety_p = numpy.percentile(subgraph, 60)
    for i, row in enumerate(subgraph):
        for j, elem in enumerate(row):
            if(elem > math.ceil(ninety_p)):
                subgraph[i][j] = None

    shuffled_subgraph = q_result[1]
    shuffled_avg_score = pandas.DataFrame(shuffled_subgraph).mean().mean()
    ninety_p_shuffled = numpy.percentile(shuffled_subgraph, 60)
    for i, row in enumerate(shuffled_subgraph):
        for j, elem in enumerate(row):
            if(elem > math.ceil(ninety_p_shuffled)):
                shuffled_subgraph[i][j] = None
    
    bs_avg_score = pandas.DataFrame(subgraph).mean().mean()
    bs_shuffled_avg_score = pandas.DataFrame(shuffled_subgraph).mean().mean()

#     print(motif, avg_score, shuffled_avg_score)
    do_not_print = db.query("UPDATE motif SET bs_average_score = " + str(bs_avg_score) + ",bs_shuffled_average_score = '" + str(bs_shuffled_avg_score) + "'  WHERE motif_string  = '" + motif + "'")
#     print(motif, avg_score,bs_avg_score, shuffled_avg_score, bs_shuffled_avg_score)
    
    